# Import libraries

In [1]:
import sys
if not 'Spatial-interpolation' in sys.path:
    sys.path += ['Spatial-interpolation']

In [ ]:
import os
# Check the current working directory
current_directory = os.getcwd()
print("Current Directory:", current_directory)
from google.colab import drive
import sys
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from torch import nn
import matplotlib.pyplot as plt
from utils import (load_nerl_data,
    get_normalized_adj,
    get_Laplace,
    calculate_random_walk_matrix,
    test_error_cap,
    test_error
)
import random
import copy
import scipy
from scipy.io import loadmat
import math
import pandas as pd
from basic_structure import D_GCN, C_GCN, K_GCN,IGNNK
from basic_process import *
from sklearn.manifold import TSNE
import os
import pandas as pd
from sklearn import linear_model
from pandas import read_csv

# Define parameters and load file

In [ ]:
n_o_n_m = 20 #sampled space dimension

h = 24 #sampled time dimension

z = 100 #hidden dimension for graph convolution

K = 1 #If using diffusion convolution, the actual diffusion convolution step is K+1

n_m = 20 #number of mask node during training

N_u = 50 #target locations, N_u locations will be deleted from the training data

Max_episode = 750 #max training episode

learning_rate = 0.0001 #the learning_rate for Adam optimizer

batch_size = 8

E_maxvalue=80

STmodel = IGNNK(h, z, K)
STmodel.load_state_dict(torch.load('Spatial-interpolation/model/IGNNK_ushcn_750iter_2023-11-26 12:25:53.pth'))

# Experiment 1: IGNNK-Dis-randsubgrh-3DGNN (Format:Metd-Adj-subgraph-Aggregation)

# Step1: Import dataset from all txt files and csv file.

In [ ]:
pathroot='/content/Spatial-interpolation/NDBC/all_stations'
PATH_ROOT = os.getcwd()
ROOT = os.path.join(PATH_ROOT, pathroot)
filenames = os.listdir(ROOT)
# Sort all files
filenames.sort()
data = []
for i in filenames:
    PATH_CSV = os.path.join(ROOT, i)
    with open(PATH_CSV, 'r') as file:
# Use splitlines() to divide contents in the documents into lists.
        content_list = file.read().splitlines()
# Transform lists into Numpy.
    content_matrix = np.array([list(map(float, line.split())) for line in content_list])
    data.append(content_matrix)
data = np.array(data).transpose(1, 2, 0)
X_raw=data

# Step2: For loop on each variable.

In [ ]:
for j in range(5,X_raw.shape[1]-1):
    #Raw data and raw adjacency matrix without mapminmax
    # X_raw=normalize_3d_array(X_raw)
    Station_info=pd.read_csv('/content/Spatial-interpolation/NDBC/Station_info.csv')
    NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
    NDBC_long=pd.DataFrame(Station_info.iloc[:,3])
    NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
    Adj_dist=adj_dist(NDBC_lat,NDBC_long)
    X_raw_0=X_raw[:,j,:]#GET the first feature
    print(X_raw_0.shape)
    # Count the num of missing values in each column
    missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
    # print results
    print("Incomplete data number in each column：", missing_counts_per_column)

    #Get the index if the value is not zero
    # Find the columns where missing values exist.
    columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

    # Get the column numbers when missing value exist.
    missing_columns = np.where(columns_with_missing_data)[0]

    # Print results
    print("The column numbers with missing values are：", missing_columns)

    #Get new data after deletion.
    #Delete those columns(Stations) if there is not any features.
    result = np.delete(X_raw_0, missing_columns, axis=1)
    result = (result - np.min(result)) / (np.max(result) - np.min(result))
    #Get new adjacency matrix after deletion.
    NDBC_long=NDBC_long.transpose()
    NDBC_long_aft=NDBC_long.drop(columns=missing_columns)
    # Reconstruct the index in the DataFrame and get the longitude after deletion.
    NDBC_long_aft = NDBC_long_aft.reset_index(drop=True)
    print(NDBC_long_aft.shape)

    NDBC_lat=NDBC_lat.transpose()
    NDBC_lat_aft=NDBC_lat.drop(columns=missing_columns)
    # Reconstruct the index in the DataFrame and get the latitude after deletion.
    NDBC_lat_aft = NDBC_lat_aft.reset_index(drop=True)
    print(NDBC_lat_aft.shape)
    #GEt the new ADJ matrix.
    Adj_dist=adj_dist(NDBC_lat_aft.transpose(),NDBC_long_aft.transpose())

    capacities=np.ones((Adj_dist.shape[0], 1))
    capacities=capacities.flatten()
    split_line1 = int(result.shape[0] * 0.7)
    training_set = result[:split_line1, :]
    test_set = result[split_line1:, :]
    rand = np.random.RandomState(0) # Fixed random output, just an example when seed = 0.
    unknow_set = rand.choice(list(range(0,result.shape[1])),N_u,replace=True)#Stations to be predicted
    unknow_set = set(unknow_set)
    full_set = set(range(0,result.shape[1]))
    know_set = full_set - unknow_set
    training_set_s = training_set[:, list(know_set)]   # get the training data in the sample time period
    A_s = Adj_dist[:, list(know_set)][list(know_set), :]
    MAE_t, RMSE_t, R2_t, nrel_ignnk_res  = test_error_cap(STmodel, unknow_set, full_set,test_set, Adj_dist,h,capacities)
    #MAE_t, RMSE_t, R2_t, nrel_ignnk_res  = test_error(STmodel, unknow_set, test_set, A_s,E_maxvalue, True)
    print('Best model in the',j, '-th column is', MAE_t, RMSE_t, R2_t)

# 